### Importing section

In [271]:
import os
import string
from nltk import tokenize
from gensim.models import Word2Vec
from nltk import word_tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
import gensim
from nltk.corpus import stopwords

### Reading the data from the folder and cleaning it.

In [272]:
sentences = []
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           #s = s.lower()
           #s = s.translate(str.maketrans('', '', string.punctuation))
           sentences.append(s)

tokensSentenceslist = []

for s  in sentences:
    wordsList = gensim.utils.simple_preprocess(s)
    filtered_words = [word for word in wordsList if word not in stopwords.words('english')]
    tokensSentenceslist.append(filtered_words)


##################### Uncomment below section for testing #########################
# print(len(sentences))
#
# for s in sentences:
#      print("The sentence is : ")
#      print(s)
#      print("-----------------------End of the sentence -------------")
#
# print (sentences)


# print (len(tokensSentenceslist))
# print (tokensSentenceslist)


In [273]:
print (tokensSentenceslist[0])

['advertising', 'television', 'becoming', 'clever']


In [247]:
testSentences = gensim.utils.simple_preprocess("This is just a test and i don't know")

print (testSentences)

['this', 'is', 'just', 'test', 'and', 'don', 'know']


In [274]:
print (len(sentences))
print(sentences[0:5])

38715
['Advertising in Television is becoming more and more clever.', 'Lots of those ads invade our subconsciousness and try to make us buy the product even if we do not actually need it.', "When it comes to children, lots of them don't have enough money to afford these products.", "They don't even understand the concept of money.", 'Even if they wanted it, they can not get the thing they were told they need.']


Generating the Word2Vec Model

In [275]:
#model = Word2Vec(tokensSentenceslist, min_count=1)

#model = Word2Vec(tokensSentenceslist, vector_size=50, min_count=1, sg=1)
#model = Word2Vec(sentences=tokensSentenceslist, vector_size=100, workers=1, seed=42)

model = Word2Vec(window=10, min_count=2,workers=6,vector_size=50,seed=42,sg=0)
model.build_vocab(tokensSentenceslist, progress_per=1000)
model.train(tokensSentenceslist, total_examples=model.corpus_count, epochs=model.epochs)


##################### Uncomment below section for testing #########################


# print(list(model.wv.index_to_key))
# print(len(list(model.wv.index_to_key)))

(1313264, 1737390)

In [276]:
model.corpus_count
#model.epochs

38715

In [281]:
#model.wv.most_similar("television")
model.wv.most_similar("argument")
#model.wv.similarity("tv","television")

[('point', 0.9456915855407715),
 ('reason', 0.9341050386428833),
 ('side', 0.9020331501960754),
 ('aspect', 0.8889020681381226),
 ('agree', 0.8727644085884094),
 ('support', 0.868994414806366),
 ('reasons', 0.8553298711776733),
 ('aspects', 0.8501182794570923),
 ('disagree', 0.8476272821426392),
 ('personally', 0.8415126204490662)]

### Vectorizing each sentence using the avg of the Word embidings of each word

In [282]:
def vectorize(list_of_docs, model, strategy):
    """Generate vectors for list of documents using a Word Emx`bedding.

    Args:
        list_of_docs: List of documents.
        model: Gensim Word Embedding.
        strategy: Aggregation strategy ("average", or "min-max".)

    Raises:
        ValueError: If the strategy is other than "average" or "min-max".

    Returns:
        List of vectors.
    """
    features = []
    size_output = model.vector_size
    embedding_dict = model

    if strategy == "min-max":
        size_output *= 2

    if hasattr(model, "wv"):
        embedding_dict = model.wv

    for tokens in list_of_docs:
        zero_vector = np.zeros(size_output)
        vectors = []
        for token in tokens:
            if token in embedding_dict:
                try:
                    vectors.append(embedding_dict[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            if strategy == "min-max":
                min_vec = vectors.min(axis=0)
                max_vec = vectors.max(axis=0)
                features.append(np.concatenate((min_vec, max_vec)))
            elif strategy == "average":
                avg_vec = vectors.mean(axis=0)
                features.append(avg_vec)
            else:
                raise ValueError(f"Aggregation strategy {strategy} does not exist!")
        else:
            features.append(zero_vector)
    return features

### Apply the function above

In [283]:
vectorized_docs = vectorize(tokensSentenceslist, model=model, strategy="average")

Test

In [284]:
len(vectorized_docs), len(vectorized_docs[0])
print(model.wv["argument"])

print("#######################################################")
print(vectorized_docs[0])

[ 6.0167569e-01 -7.7913541e-01  9.7707295e-01  8.8773452e-04
 -3.0638605e-01 -2.1325390e-01 -1.0785919e+00 -2.6945767e-01
  2.0254721e-01  2.7150404e-01  6.7686290e-01  9.5995587e-01
  8.0758250e-01  1.0841374e+00  9.6509784e-01  8.8220216e-02
  8.1530887e-01 -1.7672682e+00 -6.3352096e-01  5.6337196e-01
 -6.2989330e-01 -2.4646464e-01  5.2156770e-01  2.5482684e-01
  5.2680212e-01  2.7302524e-01 -3.9422449e-01 -6.4586878e-01
  8.5464883e-01  5.9058088e-01 -2.6026830e-01 -1.0212562e+00
 -5.1818579e-02 -4.1434370e-02  6.6938829e-01  5.3291500e-01
 -5.5190966e-02 -3.0560575e+00 -5.5581880e-01  3.9764848e-01
 -1.5702914e-01 -5.5771077e-01  1.3291311e+00 -9.2295527e-01
 -5.7646757e-01  4.1336891e-01 -1.7303368e-01  7.0126784e-01
  9.2575854e-01 -1.5619569e-01]
#######################################################
[ 0.13167055 -0.9581087   0.18906346 -0.02515565 -0.16255502 -0.37995827
  0.11477079  0.11843309  0.07814962  0.42332467  0.01161281  0.32912475
  0.08110611  1.1597311   0.327907

### Kmeans algorithm with mini batch

In [285]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

### Applying the Kmeans algorithm

In [361]:
clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=15, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "tokens": [" ".join(text) for text in tokensSentenceslist],
    "cluster": cluster_labels
})

For n_clusters = 15
Silhouette coefficient: 0.13
Inertia:90663.38629044882
Silhouette values:
    Cluster 8: Size:163 | Avg:1.00 | Min:0.86 | Max: 1.00
    Cluster 2: Size:492 | Avg:0.62 | Min:0.17 | Max: 0.66
    Cluster 0: Size:1455 | Avg:0.28 | Min:-0.02 | Max: 0.47
    Cluster 6: Size:551 | Avg:0.25 | Min:-0.19 | Max: 0.48
    Cluster 1: Size:4929 | Avg:0.21 | Min:-0.03 | Max: 0.44
    Cluster 11: Size:805 | Avg:0.14 | Min:-0.11 | Max: 0.39
    Cluster 7: Size:3569 | Avg:0.14 | Min:-0.05 | Max: 0.35
    Cluster 13: Size:3009 | Avg:0.12 | Min:-0.06 | Max: 0.34
    Cluster 9: Size:2435 | Avg:0.11 | Min:-0.08 | Max: 0.33
    Cluster 4: Size:3555 | Avg:0.10 | Min:-0.06 | Max: 0.30
    Cluster 3: Size:5917 | Avg:0.09 | Min:-0.08 | Max: 0.30
    Cluster 12: Size:1679 | Avg:0.09 | Min:-0.14 | Max: 0.36
    Cluster 10: Size:3001 | Avg:0.08 | Min:-0.11 | Max: 0.32
    Cluster 5: Size:5273 | Avg:0.08 | Min:-0.09 | Max: 0.28
    Cluster 14: Size:1882 | Avg:0.08 | Min:-0.12 | Max: 0.31


### Evaluate top terms of the cluster

In [287]:
print("Top terms per cluster (based on centroids):")
for i in range(10): # number of cluster k should be put here!!
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=10)
    #print(clustering.cluster_centers_[i])
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Top terms per cluster (based on centroids):
Cluster 0: nutshell allowed difficult opinion forbidden allowd childern concluding anyway ages 
Cluster 1: behave others theire addicted bored freetime entertaint entertain forget healthy 
Cluster 2: pour water cat catching mcdonald shoes machine die sie hell 
Cluster 3: difference furthermore canâ worse must arenâ helpful childen donâ handle 
Cluster 4: advertises necessary firstly advertisings anyway difficult matter adversting complicated however 
Cluster 5: probably tell someone anything articles automatically feeling stop mad annoy 
Cluster 6: mean means actually impress course sure exactly ones makes cannot 
Cluster 7: anyway shouldnâ musst controll letting sometimes whatch put arenâ childen 
Cluster 8: discuss answer dicuss write following reasons theme sides discussed discussion 
Cluster 9: cant worse partents theirselves childen cases addition possibility chance partens 


In [288]:
from collections import Counter

for i in range(10):
    tokens_per_cluster = ""
    most_frequent = Counter(" ".join(df_clusters.query(f"cluster == {i}")["tokens"]).split()).most_common(5)
    for t in most_frequent:
        tokens_per_cluster += f"{t[0]}({str(t[1])}) "
    print(f"Cluster {i}: {tokens_per_cluster}")

Cluster 0: children(3108) young(2356) advertising(2148) television(1847) directed(1647) 
Cluster 1: children(629) learn(557) play(547) time(440) outside(433) 
Cluster 2: sasageyo(1092) die(654) qualitã(325) zeit(325) sie(324) 
Cluster 3: children(3045) things(1078) parents(907) get(788) young(786) 
Cluster 4: children(4398) advertising(2718) young(2248) television(2007) think(1158) 
Cluster 5: parents(2150) want(1793) buy(1532) children(1397) child(1159) 
Cluster 6: children(925) products(692) money(689) make(571) companies(560) 
Cluster 7: tv(2194) watch(2138) children(2013) television(1349) watching(885) 
Cluster 8: statement(331) schreibaufgabe(293) agree(260) following(211) disagree(191) 
Cluster 9: children(1161) advertising(631) television(593) like(536) would(536) 


### Retrieve a random sample of documents for a given cluster


In [289]:
for i,t in enumerate(df_clusters.query(f"cluster == {0}").sample(10).iterrows()):
    print(t[1]["text"])
    print("-------------")

In the following text I am going to state my opinion on the statement: Television advertising directed toward young children (aged two to five) should not be allowed.
-------------
Televison advertising towards kids between the age of two to five are in my opinion both good and bad.
-------------
Then we come to the next negative point by television advertising toward young children.
-------------
Should television advertising directed toward young children be allowed or is it just a dumb thing?
-------------
Firstly, Television toward young children influences them in various ways.
-------------
I came to the conclusion that television advertising aimed towards young children should be prohibited.
-------------
The following essay talks about television advertising to young children.
-------------
There is no definition of advertising for a two year old.
-------------
Not only the advertising.
-------------
In this essay, the question about banning television ads directed toward young

In [362]:
df_clusters.to_csv('clusteredArgument.csv')

In [356]:
df_clusters.shape

# df_clusters.tokens[0]
# df_clusters.head()

(38715, 3)

### Most representative clusters

In [364]:
test_cluster = 0
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
# print(most_representative_docs[0])
for d in most_representative_docs[:10]:
    print(d)
    print(sentences[d])
    print("-------------")

8110
The statement "Television advertising directed toward young children (aged two or five) should not be allowed" says that television are directing advertisement towards children, what in my opinion is not okay.
-------------
8335
In conclusion I can certainly say that television and internet advertising directed toward young children (aged two to five) should not be allowed.
-------------
34248
The question whether television advertising directed toward young children, from age two to five, should be forbidden by law is controversial.
-------------
35721
This text should be helpful for parents who have young children aged two to five because here we awnser the question if television advertising directed toward young children should be allowed or not!
-------------
36294
Personally, I think that television advertising directed toward young children (aged two to five) should not be allowed, because it has a negative influence on the childhood.
-------------
18548
Today i would like t

In [293]:
#array = clustering.cluster_centers_[0]
#print(len(vectorized_docs))
#print(array)
array = vectorized_docs[120].reshape(1,-1)
convertedArray = array.astype(float)
# vectorized_docs[i] = sentences[i] it is the same
clustering.predict(convertedArray)

array([1])

In [191]:
#print(array.reshape(1,-1))

print(array)

print(convertedArray)

[[ 0.41131788 -0.11716345  0.28971842  0.5577258  -0.5772017   0.9901962
   0.96931005 -0.5613676  -0.3226952  -0.4045963   0.18726283  0.44057044
  -0.42001647 -0.4549887  -0.8095281   0.07320678 -0.20656103 -0.8354379
   0.20628421 -0.6952043  -0.29465434 -0.11770947 -0.03002735  0.10196522
   0.37290215 -0.10642844  0.10388153 -0.5029228  -0.48719856  0.58507514
  -0.10985485 -0.43433896 -0.43355358 -0.5003478  -0.255086    1.0371667
  -0.56815755 -1.0393796  -0.58956033  0.4535848  -0.05363839 -0.7777182
   1.1643625  -0.07477699 -0.07372791 -0.9933421  -0.22898763 -0.10050163
  -0.13402921 -0.31821993]]
[[ 0.41131788 -0.11716345  0.28971842  0.55772579 -0.57720172  0.99019623
   0.96931005 -0.56136757 -0.3226952  -0.4045963   0.18726283  0.44057044
  -0.42001647 -0.45498869 -0.80952811  0.07320678 -0.20656103 -0.83543789
   0.20628421 -0.69520432 -0.29465434 -0.11770947 -0.03002735  0.10196522
   0.37290215 -0.10642844  0.10388153 -0.50292277 -0.48719856  0.58507514
  -0.10985485 

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_25384\3934604238.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  convertedArray = array.astype(np.float)


### Predicting new clusters for testing

In [359]:
## testing
def vectorizeSentenceTest(sentences):
    tokensSentenceslist = []
    for s  in sentences:
        wordsList = word_tokenize(s)
        tokensSentenceslist.append(wordsList)
    return tokensSentenceslist

testTokens = vectorizeSentenceTest(["Watching tv for long hours leads to lazness"])

vectorized_docs_tesing = vectorize(testTokens, model=model, strategy="average")

def predictTest(vectorizedDocsTest):
    array = vectorizedDocsTest
    print(clustering.predict(array))
    return
#len(vectorized_docs_tesing), len(vectorized_docs_tesing[0])
#print(vectorized_docs_tesing)

predictTest(vectorized_docs_tesing)

[5]


In [324]:
from Cython import typeof

print(typeof(vectorized_docs))

# print(vectorized_docs[0])
# print(vectorized_docs[0].shape)
# print ("############################################")
#
# print(clustering.cluster_centers_[0])
# print(clustering.cluster_centers_[0].shape)
print (most_representative_docs.shape)

list
(38715,)


In [322]:
itemindex = np.where(vectorized_docs == clustering.cluster_centers_[0])

print(itemindex)

(array([], dtype=int64), array([], dtype=int64))


In [332]:
testDocs = np.sort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[0], axis=1)
)

In [334]:
print(vectorized_docs[16035])

[-0.02014501 -0.8525229  -0.04869652 -0.13764937 -0.15015027  0.050198
 -0.16530403  0.2516511  -0.17082413  0.35278627  0.10296659  0.5194729
  0.14524572  1.0872988   0.4511473   0.17673676  0.23570392 -1.1361816
 -0.38236055 -0.02064579 -0.5204541   0.09028093 -0.1568032   0.39367923
  0.9687556   0.00753525 -0.11855845  0.01320843 -0.4333722   0.24393816
 -0.30497146 -0.32613945 -0.07790004  0.1395214   0.75188285 -0.22694431
 -0.32751098 -1.3844128  -0.04305803  0.2324039  -0.29520527  0.21222845
  0.75226027 -0.36937404  0.07873303 -0.20217176  0.27925226  1.0212195
  0.86630887 -0.4531421 ]


### Testing wether the centroid are sentences or not

In [351]:
itemindex = np.where(vectorized_docs == clustering.cluster_centers_[9])

print(itemindex[0])

[]
